In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import os
from PIL import Image, ImageOps
from sklearn.metrics import f1_score, recall_score
import numpy as np
from tqdm import tqdm as tqdm
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
f"Using {device}"

'Using cuda'

In [4]:
num_diseases = 485
root_dir = "./img"

In [5]:
class ResizeWithPadAndCenter(object):
    def __init__(self, size, padding_value=0):
        self.size = size
        self.padding_value = padding_value

    def __call__(self, img):
        delta_w = max(self.size[0] - img.size[0])
        delta_h = max(self.size[1] - img.size[1])
        padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
        img = ImageOps.expand(img, padding, fill=self.padding_value)
        img = img.resize(self.size, Image.BILINEAR)
        return img

In [6]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

In [7]:
transforms = transforms.Compose([
    ResizeWithPadAndCenter((224, 224), padding_value=128)
    ,transforms.ToTensor()
    ,transforms.Normalize(mean, std)
])

datasets = datasets.ImageFolder(root_dir, transform=transforms)


In [9]:
train_size = int(0.8 * len(datasets))
test_size = len(datasets) - train_size

In [10]:
train_dataloader, test_dataloader = random_split(datasets, [train_size, test_size])

In [11]:
train_dataloader = DataLoader(train_dataloader, batch_size=64, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_dataloader, batch_size=64, shuffle=True, num_workers=4)

In [12]:
class DenseNet201(nn.Module):
    def __init__(self, num_classes = num_diseases):
            super(DenseNet201, self).__init__()
            self.densenet201 = models.densenet201(weights=models.DenseNet201_Weights.IMAGENET1K_V1)
            num_ftrs = self.densenet201.classifier.in_features
            self.densenet201.classifier = nn.Linear(num_ftrs, num_classes)
            
            for param in self.densenet201.parameters():
                param.requires_grad = False
            for param in self.densenet201.classifier.parameters():
                param.requires_grad = True

    def forward(self, x):
        return self.densenet(x)

In [19]:
model = DenseNet201().to(device)

In [25]:
optim = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

In [26]:
def check_dataloader(dataloader):
    for batch, (X, y) in enumerate(dataloader):
        for i in range(X.size(0)):
            img = X[i]
            print(f"Image {i} in batch {batch}: size {img.size()}, dtype {img.dtype}")
            if img.size() != (3, 224, 224):
                print(f"Unexpected image size: {img.size()} in batch {batch} index {i}")
            if not torch.is_floating_point(img):
                print(f"Unexpected image type: {img.dtype} in batch {batch} index {i}")

In [27]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    for batch, (X, y) in tqdm(enumerate(dataloader)):
        X, y = X.to(device), y.to(device)
        
        # Forward pass
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{len(dataloader.dataset):>5d}]")


In [28]:
def test(dataloader, model, loss_fn):
    model.eval()
    test_loss, correct = 0, 0
    all_preds, all_labels = []
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    test_loss /= len(dataloader)
    accuracy = correct / len(dataloader.dataset)
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    top1_accuracy = np.mean(np.any(all_preds.argsort(axis=1)[:, -1:] == all_labels[:, None], axis=1))
    top2_accuracy = np.mean(np.any(all_preds.argsort(axis=1)[:, -2:] == all_labels[:, None], axis=1))
    top5_accuracy = np.mean(np.any(all_preds.argsort(axis=1)[:, -5:] == all_labels[:, None], axis=1))
    
    print(f"Test Error: \n Accuracy: {(100*accuracy):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    print(f"Top-1 Accuracy: {top1_accuracy:>0.1f}")
    print(f"Top-2 Accuracy: {top2_accuracy:>0.1f}")
    print(f"Top-5 Accuracy: {top5_accuracy:>0.1f}")

In [29]:
epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optim)
    test(test_dataloader, model, loss_fn)

print("Training Complete")


Epoch 1
-------------------------------


0it [00:00, ?it/s]


TypeError: 'int' object is not iterable